# Introdução 

Nesse tutorial vamos aprender o básico sobre as etapas de uma rotina de ETL(extract, transform and load). A ideia aqui é utilizar um dataset público real para demonstrar como implementar as diferentes etapas dessa rotina. Vamos explorar alguns conceitos básicos da Engenharia de dados, como implementar rotinas para extração de arquivos e como manipular dados tabulares com o Pandas. Lembrando que toda a implementação aqui é focada em *small data*.

# O que é ETL

**ETL(Extract, transform and load)** é um processo que consiste em integrar dados de diferentes fontes buscando consolidá-los de uma maneira que facilite o processo de análise. O termo se popularizou com o surgimento das *data warehouses*, que nasceram da necessidade de centralizar diversas fontes de dados para permitir criar análises que ajudassem as empresas na tomada de decisão.

O processo consiste de três etapas:

1. *Extract*: extrair dados de uma fonte de informações, seja banco de dados, API, arquivos, etc. Para serem processados posteriormente;
2. *Transform*: nessa etapa os dados extraídos passam por uma transformação para atender os requisitos das aplicações cliente. A transormação envolve:
    - Limpar e validar os dados para garantir qualidade;
    - Transformar o formato dos dados para, por exemplo, facilitar usabilidade e busca;
    - Combinar diferentes fontes para compor as informações necessárias;
    - Aplicar regras de negócio.
3. *Load*: carregar resultado das transformações no sistema de destino, como *date warehouses* ou *data lakes*.

![](./img/etl_diagram.png)

# A fonte de dados 

Vamos trabalhar com a fonte de dados abertos da [Comissão de Valores Mobiliários](http://www.cvm.gov.br/) contendo a cotação diária dos fundos de investimentos negociados no mercado brasileiro. Essa fonte é atualizada diariamente com os dados de fechamento do dia anterior e as cotações são agrupadas por arquivos correspondentes a cada mês do ano.

Esse é um cenário bem comum em fontes de dados abertas, uma série de arquivos no formato CSV agrupando informações de acordo com a data, então a solução que vamos implementar durante o tutorial é reaproveitável para outras fontes de dados.

Primeiro é importante analisar a fonte de dados, entender como ela está estruturada, quais campos compõem o dataset e como nós podemos automatizar a coleta dos dados.

A fonte de dados contendo a cotação diária dos fundos pode ser acessada através do portal de dados abertos pelo link:

http://www.dados.gov.br/dataset/fi-doc-inf_diario

## Como automatizar o download 

Podemos observar que no site existe um link para todos os datasets dos últimos 12 meses. Primeiro precisamos analisar se existe um padrão nas urls de download dos arquivos, para isso vamos copiar alguns endereços e compará-los.

Copiei três links e vamos compará-los a seguir:

http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201907.csv

http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201910.csv

http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202003.csv

Como podemos observar existe um padrão claro nos links:

`dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_YYYYMM.csv`

# ETL

In [2]:
import pandas as pd
import requests
from tqdm import tqdm

## Extrair 

O processo de extração consiste nesse caso em fazer o download de todos os arquivos da janela de tempo q nos interessa. As etapas pra nós atingirmos esse objetivo são:

1. Automatizar a geração do nome dos arquivos: já que a única coisa que varia nos links é a data dos arquivos precisamos automatizar a geração dessas datas de acordo com a janela de tempo do nosso interesse;

2. Requisitar o arquivo: precisamos enviar uma requisição para o portal de dados abertos do arquivo que queremos fazer o download;

3. Salvar arquivo: o portal de dados abertos vai nos enviar o arquivo requisitado e precisamos salvá-lo no nosso computador.

### Gera lista de datas

In [3]:
def generate_dates(start, end, freq='M', format='%Y%m'):
    return [date.strftime(format) for date in pd.date_range(start=start, end=end, freq=freq)]

### Requisita e salva os arquivos 

In [4]:
def download_files(dates, save_dir='data'):
    saved_files = []
    
    file_base_url = 'http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{date}.csv'
    
    for date in tqdm(dates):
        url = file_base_url.format(date=date)
        
        response = requests.get(url)
        
        if response.status_code != 200:
            print('Erro {status} ao requisitar o arquivo: {file}'.format(status=response.status_code, file=url))
            continue
        
        file_save_name = '{dir}/inf_diario_fi_{date}.csv'.format(dir=save_dir, date=date)
        
        with open(file_save_name, 'wb') as f:
            f.write(response.content)
        
        saved_files.append(file_save_name)
    
    return saved_files

In [5]:
def extract(initial_date, final_date, save_dir='data'):
    date_range = generate_dates(initial_date, final_date)
    
    files = download_files(date_range, save_dir)
    return files

## Análise

Aqui vou abrir um parênteses para uma análise rápido do nosso dateframe, é claro que só a parte de exploração dos dados vale um tutorial completo, então não vou explorar muito esses aspecto aqui, mas de qualquer forma é importante ter noção de alguns pontos básicos quando se trabalha com uma fonte de dados:

- Qual o tipo de cada coluna
- Quantos valores nulos que existem
- Como estão formatados

Essa etapa não faz parte do ETL, na verdade essa exploração inicial normalmente é feita antes de construir a *pipeline* para entender a fonte de dados utilizada.

In [6]:
raw_df = pd.read_csv('data/inf_diario_fi_202001.csv', sep=';')

In [7]:
# Formato do dataframe (linhas, colunas)
raw_df.shape

(369894, 8)

In [8]:
# Quais são nossas colunas, seus tipos e se existem valores nulos
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369894 entries, 0 to 369893
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   CNPJ_FUNDO     369894 non-null  object 
 1   DT_COMPTC      369894 non-null  object 
 2   VL_TOTAL       369894 non-null  float64
 3   VL_QUOTA       369894 non-null  float64
 4   VL_PATRIM_LIQ  369894 non-null  float64
 5   CAPTC_DIA      369894 non-null  float64
 6   RESG_DIA       369894 non-null  float64
 7   NR_COTST       369894 non-null  int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 22.6+ MB


In [9]:
raw_df.head()

,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,00.017.024/0001-53,2020-01-02,1132491.66,27.225023,1123583.00,0.0,0.0,1
1,00.017.024/0001-53,2020-01-03,1132685.12,27.224496,1123561.25,0.0,0.0,1
2,00.017.024/0001-53,2020-01-06,1132881.43,27.225564,1123605.31,0.0,0.0,1
3,00.017.024/0001-53,2020-01-07,1133076.85,27.226701,1123652.24,0.0,0.0,1
4,00.017.024/0001-53,2020-01-08,1132948.59,27.227816,1123698.26,0.0,0.0,1


## Transformar 

Nessa etapa é necessário consolidar todas as informações extraídas, aplicar os tratamentos e regras de negócio que fazem sentido para a aplicação cliente, defini alguns objetivos para nos orientar durante essa etapa:

1. Consolidar todos os arquivos em um *dataframe*;
2. Transformar tipo da coluna de data(`DT_COMPTC`) para `datetime`;
3. Manter somente fundos com mais de `1000` cotistas;
4. Manter somente informações sobre: data, CNPJ do fundo e valor da cota;
5. Mudar o formato do dataframe para:

|            | 00.017.024/0001-53 | 97.929.213/0001-34 | 00.068.305/0001-35 | ... |
|------------|--------------------|--------------------|--------------------|-----|
| 2020-01-02 | 27.225023          | 27.112737          | 1.733476e+08       | ... |
| 2020-01-03 | 27.224496          | 27.115661          | 6.611408e+07       | ... |
| ...        | ...                | ...                | ...                | ... |

### 1. Consolida todos os arquivos 

In [10]:
def concat_csvs(file_list, sep=','):
    df_list = [pd.read_csv(file, sep=sep) for file in file_list]
    return pd.concat(df_list)

### 2. Converte coluna para tipo data 

In [11]:
def convert_col_to_datetime(df, target_col, format='%Y-%m-%d'):
    df[target_col] = pd.to_datetime(df[target_col], format=format)

### 3. Filtra por quantidade de cotistas

In [12]:
def filter_by_shareholders(df, min_value, date_col='DT_COMPTC', shareholders_col='NR_COTST', id_col='CNPJ_FUNDO'):
    # Filtra linhas pela última data do período para avaliar a quantidade de cotistas
    df_final_date = df[df[date_col] == df[date_col].max()]
    valid_ids = df_final_date.query('{} >= @min_value'.format(shareholders_col))[id_col]
    return df.query('{} in @valid_ids'.format(id_col))

### 4. Altera formato do dataframe 

In [13]:
def shift_dataframe_format(df, date_col='DT_COMPTC', id_col='CNPJ_FUNDO', quote_col='VL_QUOTA'):
    result = pd.DataFrame(data={date_col: df[date_col].sort_values().unique()})
    group_ids = df.groupby(id_col)
    
    for idx in tqdm(group_ids.groups):
        group = group_ids.get_group(idx)
        result = pd.merge(result, group[[date_col, quote_col]], on=date_col, how='left').rename(columns={quote_col: idx})
    
    result.set_index(date_col, inplace=True)
    return result

In [14]:
def transform(file_list):
    csv_sep = ';'
    date_col = 'DT_COMPTC'
    min_shareholders = 1000
    
    raw_df = concat_csvs(file_list, sep=csv_sep)
    convert_col_to_datetime(raw_df, date_col)
    filtered_by_shareholders = filter_by_shareholders(raw_df, min_shareholders)
    
    return shift_dataframe_format(filtered_by_shareholders)

## Carregar 

Para etapa de carregamento vamos exportar o resultado em formato CSV e também vou demostrar como subir esse arquivo no [S3 da Amazon](https://aws.amazon.com/pt/s3/), que é um serviço de armazenamento de objetos na cloud, comumente usado como *data lake*.

In [15]:
def load(df, file_path):
    df.to_csv(file_path)
    
    return file_path

**Bônus**: Essa função utiliza do SDK do AWS para fazer o upload do arquivo resultante em um *bucket* no S3 da AWS, não vou entrar em detalhes sobre a configuração dessa ferramenta, mas se tiver interesse em saber mais a [documentação do boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#installation) tem um passo a passo sobre como utilizar o SDK.

In [25]:
import boto3

def upload_s3(file_path, remote_file_name, bucket):
    s3 = boto3.resource('s3')
    data = open(file_path, 'rb')
    s3.Bucket(bucket).put_object(Key=remote_file_name, Body=data)

# *Pipeline* completa

In [16]:
def pipeline(initial_date, final_date, save_file_path):
    file_list = extract(initial_date, final_date)
    
    result = transform(file_list)
    
    return load(result, save_file_path)

In [17]:
pipeline('2020-01', '2020-03', 'result.csv')

100%|██████████| 1113/1113 [01:21<00:00, 13.61it/s]


'result.csv'

In [27]:
upload_s3('result.csv', 'result.csv', 'pythonbrasil')

# Conclusão

Durante o tutorial passamos por todas as etapas do processo ETL, claro que a solução que implementamos aqui é simples e trabalha com um pequeno volume de dados, a partir do momento em que o volume de dados aumenta é preciso buscar ferramentas otimizadas, mas o processo no geral continua o mesmo. E para levar essa *pipeline* para produção o que falta? 

Em produção é importante utilizar ferramentas que paralelizem a execução das tarefas e que também sejam capazes de lidar com falhas durante o processo, no caso de *pipelines* que realizam processamento em lote(como a que nós implementamos), temos por exemplo ferramentas como:
- [Apache Airflow](https://airflow.apache.org/)
- [Luigi](https://github.com/spotify/luigi)
- [Apache Beam](https://beam.apache.org/)
- [Prefect](https://www.prefect.io/)

Essa mesma *pipeline* que nós implementamos foi a primeira versão do que eu fiz para alimentar o [fundos.sharke.com.br](fundos.sharke.com.br), hoje está bem mais complexa e roda no [Apache Airflow](https://airflow.apache.org/):

![](img/airflow_dag.png)